In [ ]:
from selenium import webdriver
from webdriver_auto_update import check_driver
check_driver(r'C:\Users\Tahsin Sayed\Downloads\chromedriver_win32')
import pandas as pd  
pd.options.mode.chained_assignment = None
import string
import time

In [ ]:
folder_path = r'C:\Users\Tahsin Sayed\Desktop\Image_to_text\\'
df = pd.read_csv(folder_path+ 'company_list.csv')
df

In [ ]:
for i in df.index:
    x = df['NAMEOFISSUER'][i]
    x = ''.join('' if c in string.punctuation else c for c in x)
    df['Company_name'][i] = x.rsplit(' ', 1)[0]

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\Tahsin Sayed\\AppData\\Local\\Google\\Chrome\\User Data")
driver = webdriver.Chrome(chrome_options=options)

In [ ]:
driver.get('https://kingscrowd.com/companies/search/')

In [ ]:
input_placeholder = driver.find_element_by_id('ratings_searchbox')
for i in df.index:
    input_placeholder.clear()
    input_placeholder.send_keys(df['Company_name'][i])
    time.sleep(2)
    table = pd.read_html(driver.page_source)[0]
    if i==10:
        break
    
    if table['company'][0] == 'No matching records found':
        # There is no data for this company.
        df['Progress'][i] = 'Data Not Found'
        continue
    
    #Check start date. 
    table['start date'] = pd.to_datetime(table['start date'])
    table['close date'] = pd.to_datetime(table['close date'])
    instances = table[table['start date'].dt.year == 2018].shape[0]

    #If Multiple Instance, need to check that manually. 
    if instances > 1:
        df['Progress'][i] = 'Multiple Data Found.'
        continue
    if instances == 0:
        df['Progress'][i] = 'No Data in This Year.'
        continue
    if instances == 1:
        '''
        Here remains the code. 

        '''
        pass

In [ ]:

table['start date'] = pd.to_datetime(table['start date'])
table['start date']

In [ ]:
table[table['start date'].dt.year == 2018].shape[0]

In [ ]:
df.head(10)

In [ ]:
table = pd.read_html(driver.page_source)[0]

In [ ]:
table

In [ ]:
table['start date'] = pd.to_datetime(table['start date'])
table['close date'] = pd.to_datetime(table['close date'])
table[table['start date'].dt.year == 2030]

In [ ]:
table['start date'].dt.year

In [ ]:
table['start date'].rsplit('/', 1)[-1]


In [ ]:
driver.find_element_by_id('ratings_searchbox').clear()

In [ ]:
driver.find_element_by_id('ratings_searchbox').send_keys('HEllo')

In [ ]:
inputplaceholder = [x for x in driver.find_elements_by_xpath['//input'] if x.get_attribute('id') == 'ratings_searchbox'][0]
inputplaceholder.send_keys('Hello')